# Deep Learning 101 - Keras

In [2]:
# Setup
import os
import matplotlib.pyplot as plt
import numpy as np

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [6]:
'''This script goes along the blog post
"Building powerful image classification models using very little data"
from blog.keras.io.
It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

from keras.models import load_model
from keras.preprocessing import image


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 25
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
# Existem dois tipos de modelos, porém o Sequencial é o mais comum.

#Aqui adicionamos uma camada convulacional à rede. Declaramos que ela deve extrair 32 filtros
#e que os kernels serão uma matriz 3x3.
model.add(Conv2D(32, (3, 3), input_shape=input_shape))


# Em seguida, uma camada de neurônios de ativação, RELU é um tipo de função de ativação.
model.add(Activation('relu'))

#A camada de Max Pooling é adicionada e tem como finalidade destacar apenas os filtros gerados 
#com maior valor de ativação. Ex.: Detectar Features mais relevantes
model.add(MaxPooling2D(pool_size=(2, 2)))

#Adicionamos mais camadas Convulacionais, de Ativação e de Max Pooling
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#Neste ponto, nosso modelo tem como output um mapa 3D de features! (Traços, Bordas, Retas) - Pequenas partes da imagem que 
#são comuns à cada classe.

# Por fim, utilizamos a função Flatten, para converter nosso mapa 3D de Features em um Vetor de 1 Dimensão
# (Pense em um array)
# Adicionamos também as camadas Dense (Totalmente Conectadas) e mais funções de ativação ao modelo.
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

#Para cada tipo de problema, existe uma loss function, uma função de otimização e as métricas mais adequadas.
#Esses elementos são parâmetros para a compilação do modelo
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

#Após definir o modelo da rede e compilar, criamos instâncias de geradores de dados 
#(Elementos que vão ler cada uma das images de imput)
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save('first_try.h5')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/25
125/125 [==============================] - 41s 325ms/step - loss: 0.7074 - acc: 0.5285 - val_loss: 0.6795 - val_acc: 0.6112
Epoch 2/25
125/125 [==============================] - 37s 300ms/step - loss: 0.6658 - acc: 0.6105 - val_loss: 0.6309 - val_acc: 0.6881
Epoch 3/25
125/125 [==============================] - 37s 298ms/step - loss: 0.6341 - acc: 0.6565 - val_loss: 0.6034 - val_acc: 0.6578
Epoch 4/25
125/125 [==============================] - 37s 296ms/step - loss: 0.6120 - acc: 0.6735 - val_loss: 0.5882 - val_acc: 0.6806
Epoch 5/25
125/125 [==============================] - 38s 301ms/step - loss: 0.6000 - acc: 0.6915 - val_loss: 0.5380 - val_acc: 0.7113
Epoch 6/25
125/125 [==============================] - 37s 300ms/step - loss: 0.5772 - acc: 0.7150 - val_loss: 0.5280 - val_acc: 0.7437
Epoch 7/25
125/125 [==============================] - 38s 301ms/step - loss: 0.5649 - acc: 0.7140 - val_lo

In [15]:
classes = train_generator.class_indices
print(classes)

{'cats': 0, 'dogs': 1}


In [22]:
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(150, 150))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

In [40]:
 # load model
model = load_model("first_try.h5")

# image path
img_path_1 = 'test/tchalla1.jpg'    # cat
img_path_2 = 'test/dog1.jpg'    # dog

# load a single image
new_image_1 = load_image(img_path_1, True)
new_image_2 = load_image(img_path_2, True)


# check prediction
pred_1 = model.predict(new_image_1)
pred_2 = model.predict(new_image_2)

In [41]:
print(pred_1)
print(pred_2)


[[0.16985281]]
[[0.9496022]]


In [35]:
if pred_1[0][0] >= 0.5:
    result_1 = 'dog'
else:
    result_1 = 'cat'

print(result_1)

cat


In [38]:
if pred_2[0][0] >= 0.5:
    result_2 = 'dog'
else:
    result_2 = 'cat'

print(result_2)

dog
